<a href="https://colab.research.google.com/github/kapoorkhushi03/TRY-IT_ON/blob/main/GeometricMatchingModule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install TensorFlow
!pip install tensorflow

# Install TensorFlow Addons for additional functionalities (optional)
!pip install tensorflow-addons

# Install OpenCV for image processing
!pip install opencv-python-headless


In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# For loading datasets and creating data loaders
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import VGG16  # or another pre-trained model
from tensorflow.keras import layers, models

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# Load images
img1_path = '/content/image_cloth.jpg'
img2_path = '/content/image_mask2.jpg'

img1 = cv2.imread(img1_path)
img2 = cv2.imread(img2_path)

# Convert images to grayscale
img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# Resize images
img1_resized = cv2.resize(img1_gray, (256, 256))
img2_resized = cv2.resize(img2_gray, (256, 256))

# Normalize images
scaler = MinMaxScaler()
img1_normalized = scaler.fit_transform(img1_resized)
img2_normalized = scaler.transform(img2_resized)

# Convert to TensorFlow tensors
img1_tensor = tf.convert_to_tensor(img1_normalized, dtype=tf.float32)
img2_tensor = tf.convert_to_tensor(img2_normalized, dtype=tf.float32)

# Add batch and channel dimensions
img1_tensor = tf.expand_dims(img1_tensor, axis=0)  # Shape [1, 256, 256]
img1_tensor = tf.expand_dims(img1_tensor, axis=-1)  # Shape [1, 256, 256, 1]

img2_tensor = tf.expand_dims(img2_tensor, axis=0)  # Shape [1, 256, 256]
img2_tensor = tf.expand_dims(img2_tensor, axis=-1)  # Shape [1, 256, 256, 1]

# Print shapes to verify
print(f'img1_tensor shape: {img1_tensor.shape}')
print(f'img2_tensor shape: {img2_tensor.shape}')


img1_tensor shape: (1, 256, 256, 1)
img2_tensor shape: (1, 256, 256, 1)


In [ ]:
import tensorflow as tf
import cv2
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load and preprocess images
img1_path = '/content/image_cloth.jpg'
img2_path = '/content/image_mask2.jpg'

img1 = cv2.imread(img1_path)
img2 = cv2.imread(img2_path)

# Convert images to grayscale
img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# Resize images
img1_resized = cv2.resize(img1_gray, (256, 256))
img2_resized = cv2.resize(img2_gray, (256, 256))

# Normalize images
scaler = MinMaxScaler()
img1_normalized = scaler.fit_transform(img1_resized)
img2_normalized = scaler.transform(img2_resized)

# Convert to TensorFlow tensors
img1_tensor = tf.convert_to_tensor(img1_normalized, dtype=tf.float32)
img2_tensor = tf.convert_to_tensor(img2_normalized, dtype=tf.float32)

# Add batch and channel dimensions
img1_tensor = tf.expand_dims(img1_tensor, axis=0)  # Shape [1, 256, 256]
img1_tensor = tf.expand_dims(img1_tensor, axis=-1)  # Shape [1, 256, 256, 1]

img2_tensor = tf.expand_dims(img2_tensor, axis=0)  # Shape [1, 256, 256]
img2_tensor = tf.expand_dims(img2_tensor, axis=-1)  # Shape [1, 256, 256, 1]

# Create a dataset
train_dataset = tf.data.Dataset.from_tensor_slices((img1_tensor, img1_tensor)).batch(1)

# Define the Autoencoder model
class Autoencoder(tf.keras.Model):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Conv2D(16, kernel_size=3, strides=2, padding='same', activation='relu'),  # [B, 16, 128, 128]
            tf.keras.layers.Conv2D(32, kernel_size=3, strides=2, padding='same', activation='relu'),  # [B, 32, 64, 64]
            tf.keras.layers.Conv2D(64, kernel_size=3, strides=2, padding='same', activation='relu'),  # [B, 64, 32, 32]
            tf.keras.layers.Conv2D(128, kernel_size=3, strides=2, padding='same', activation='relu')  # [B, 128, 16, 16]
        ])
        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Conv2DTranspose(64, kernel_size=3, strides=2, padding='same', activation='relu'),  # [B, 64, 32, 32]
            tf.keras.layers.Conv2DTranspose(32, kernel_size=3, strides=2, padding='same', activation='relu'),  # [B, 32, 64, 64]
            tf.keras.layers.Conv2DTranspose(16, kernel_size=3, strides=2, padding='same', activation='relu'),  # [B, 16, 128, 128]
            tf.keras.layers.Conv2DTranspose(1, kernel_size=3, strides=2, padding='same', activation='sigmoid')  # [B, 1, 256, 256]
        ])

    def call(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

    def encode(self, x):
        return self.encoder(x)

# Initialize model
autoencoder = Autoencoder()

# Define loss function and optimizer
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Compile the model with the loss function and optimizer
autoencoder.compile(optimizer=optimizer, loss=loss_fn)

# Train the autoencoder
history = autoencoder.fit(
    train_dataset,
    epochs=10  # Set the number of epochs you want
)

# Print training history
print(history.history)


Epoch 1/10
1/1 [==============================] - 8s 8s/step - loss: 0.1762
Epoch 2/10
1/1 [==============================] - 0s 78ms/step - loss: 0.1759
Epoch 3/10
1/1 [==============================] - 0s 76ms/step - loss: 0.1753
Epoch 4/10
1/1 [==============================] - 0s 91ms/step - loss: 0.1744
Epoch 5/10
1/1 [==============================] - 0s 100ms/step - loss: 0.1727
Epoch 6/10
1/1 [==============================] - 0s 115ms/step - loss: 0.1699
Epoch 7/10
1/1 [==============================] - 0s 99ms/step - loss: 0.1655
Epoch 8/10
1/1 [==============================] - 0s 65ms/step - loss: 0.1586
Epoch 9/10
1/1 [==============================] - 0s 78ms/step - loss: 0.1490
Epoch 10/10
1/1 [==============================] - 0s 59ms/step - loss: 0.1372
{'loss': [0.1762121021747589, 0.175878643989563, 0.1753210723400116, 0.17435383796691895, 0.17269492149353027, 0.16992491483688354, 0.16545391082763672, 0.15861624479293823, 0.14897778630256653, 0.1372307687997818]}


In [ ]:
import tensorflow as tf

# Define the Autoencoder model
class Autoencoder(tf.keras.Model):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # Define the encoder part
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Conv2D(16, kernel_size=3, strides=2, padding='same', activation='relu'),  # [B, 16, 128, 128]
            tf.keras.layers.Conv2D(32, kernel_size=3, strides=2, padding='same', activation='relu'),  # [B, 32, 64, 64]
            tf.keras.layers.Conv2D(64, kernel_size=3, strides=2, padding='same', activation='relu'),  # [B, 64, 32, 32]
            tf.keras.layers.Conv2D(128, kernel_size=3, strides=2, padding='same', activation='relu')  # [B, 128, 16, 16]
        ])
        # Define the decoder part
        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Conv2DTranspose(64, kernel_size=3, strides=2, padding='same', activation='relu'),  # [B, 64, 32, 32]
            tf.keras.layers.Conv2DTranspose(32, kernel_size=3, strides=2, padding='same', activation='relu'),  # [B, 32, 64, 64]
            tf.keras.layers.Conv2DTranspose(16, kernel_size=3, strides=2, padding='same', activation='relu'),  # [B, 16, 128, 128]
            tf.keras.layers.Conv2DTranspose(1, kernel_size=3, strides=2, padding='same', activation='sigmoid')  # [B, 1, 256, 256]
        ])

    def call(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

    def encode(self, x):
        return self.encoder(x)

# Initialize the model
autoencoder = Autoencoder()

# Define loss function and optimizer
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Define the training loop
num_epochs = 50
for epoch in range(num_epochs):
    with tf.GradientTape() as tape:
        # Forward pass
        outputs = autoencoder(img1_tensor, training=True)
        # Compute loss
        loss = loss_fn(img1_tensor, outputs)
    # Compute gradients
    gradients = tape.gradient(loss, autoencoder.trainable_variables)
    # Update weights
    optimizer.apply_gradients(zip(gradients, autoencoder.trainable_variables))

    # Print the loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss:.4f}')


Epoch [10/50], Loss: 0.1672
Epoch [20/50], Loss: 0.1247
Epoch [30/50], Loss: 0.0992
Epoch [40/50], Loss: 0.0839
Epoch [50/50], Loss: 0.0717


In [ ]:
import tensorflow as tf
import cv2
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Define the Autoencoder model
class Autoencoder(tf.keras.Model):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Conv2D(16, kernel_size=3, strides=2, padding='same', activation='relu'),  # [B, 16, 128, 128]
            tf.keras.layers.Conv2D(32, kernel_size=3, strides=2, padding='same', activation='relu'),  # [B, 32, 64, 64]
            tf.keras.layers.Conv2D(64, kernel_size=3, strides=2, padding='same', activation='relu'),  # [B, 64, 32, 32]
            tf.keras.layers.Conv2D(128, kernel_size=3, strides=2, padding='same', activation='relu')  # [B, 128, 16, 16]
        ])
        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Conv2DTranspose(64, kernel_size=3, strides=2, padding='same', activation='relu'),  # [B, 64, 32, 32]
            tf.keras.layers.Conv2DTranspose(32, kernel_size=3, strides=2, padding='same', activation='relu'),  # [B, 32, 64, 64]
            tf.keras.layers.Conv2DTranspose(16, kernel_size=3, strides=2, padding='same', activation='relu'),  # [B, 16, 128, 128]
            tf.keras.layers.Conv2DTranspose(1, kernel_size=3, strides=2, padding='same', activation='sigmoid')  # [B, 1, 256, 256]
        ])

    def call(self, x, training=False):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

    def encode(self, x):
        return self.encoder(x)

# Initialize the model
autoencoder = Autoencoder()

# Define loss function and optimizer
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Compile the model with the loss function and optimizer
autoencoder.compile(optimizer=optimizer, loss=loss_fn)

# Load and preprocess images
img1_path = '/content/image_cloth.jpg'
img2_path = '/content/image_mask2.jpg'

img1 = cv2.imread(img1_path)
img2 = cv2.imread(img2_path)

# Convert images to grayscale
img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# Resize images
img1_resized = cv2.resize(img1_gray, (256, 256))
img2_resized = cv2.resize(img2_gray, (256, 256))

# Normalize images
scaler = MinMaxScaler()
img1_normalized = scaler.fit_transform(img1_resized)
img2_normalized = scaler.transform(img2_resized)

# Convert to TensorFlow tensors
img1_tensor = tf.convert_to_tensor(img1_normalized, dtype=tf.float32)
img2_tensor = tf.convert_to_tensor(img2_normalized, dtype=tf.float32)

# Add batch and channel dimensions
img1_tensor = tf.expand_dims(img1_tensor, axis=0)  # Shape [1, 256, 256]
img1_tensor = tf.expand_dims(img1_tensor, axis=-1)  # Shape [1, 256, 256, 1]

img2_tensor = tf.expand_dims(img2_tensor, axis=0)  # Shape [1, 256, 256]
img2_tensor = tf.expand_dims(img2_tensor, axis=-1)  # Shape [1, 256, 256, 1]

# Assuming you have trained the model before this step
# Extract features from the images
features_img1 = autoencoder.encode(img1_tensor).numpy()
features_img2 = autoencoder.encode(img2_tensor).numpy()

print(features_img1.shape)  # Should print something like [1, 16, 16, 128]
print(features_img2.shape)  # Should print something like [1, 16, 16, 128]

# If you need to remove the batch and channel dimensions
features_img1 = np.squeeze(features_img1)  # [16, 16, 128]
features_img2 = np.squeeze(features_img2)  # [16, 16, 128]

print(features_img1.shape)  # Should print [16, 16, 128]
print(features_img2.shape)  # Should print [16, 16, 128]


(1, 16, 16, 128)
(1, 16, 16, 128)
(16, 16, 128)
(16, 16, 128)


In [ ]:
import tensorflow as tf

class FeatureCorrelation(tf.keras.layers.Layer):
    def __init__(self):
        super(FeatureCorrelation, self).__init__()

    def call(self, feature_A, feature_B):
        # Get the batch size, channels, height, and width
        b, h, w, c = tf.shape(feature_A)[0], tf.shape(feature_A)[1], tf.shape(feature_A)[2], tf.shape(feature_A)[3]

        # Reshape and transpose feature_A and feature_B
        feature_A = tf.reshape(feature_A, [b, h*w, c])  # [B, H*W, C]
        feature_B = tf.reshape(feature_B, [b, c, h*w])  # [B, C, H*W]
        feature_A = tf.transpose(feature_A, [0, 2, 1])  # [B, C, H*W]

        # Compute correlation
        correlation = tf.matmul(feature_A, feature_B)  # [B, H*W, H*W]

        # Reshape to [B, H, W, H, W]
        correlation = tf.reshape(correlation, [b, h, w, h, w])

        return correlation
        print("Feature Correlation shape:", correlation.shape)


In [ ]:
import tensorflow as tf

class FeatureRegression(tf.keras.layers.Layer):
    def __init__(self):
        super(FeatureRegression, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(128, kernel_size=3, strides=1, padding='same', activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(64, kernel_size=3, strides=1, padding='same', activation='relu')
        self.conv3 = tf.keras.layers.Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu')
        self.conv4 = tf.keras.layers.Conv2D(6, kernel_size=3, strides=1, padding='same')

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        return x
        print("Feature Regression output shape:", output.shape)



In [ ]:
import tensorflow as tf
import numpy as np

# Define the FeatureCorrelation class
class FeatureCorrelation(tf.keras.layers.Layer):
    def __init__(self):
        super(FeatureCorrelation, self).__init__()

    def call(self, feature_A, feature_B):
        # No need to squeeze here as the dimensions are already correct
        b, h, w, c = tf.shape(feature_A)[0], tf.shape(feature_A)[1], tf.shape(feature_A)[2], tf.shape(feature_A)[3]
        feature_A = tf.reshape(feature_A, [b, h*w, c])  # [B, H*W, C]
        feature_B = tf.reshape(feature_B, [b, c, h*w])  # [B, C, H*W]
        # feature_A = tf.transpose(feature_A, [0, 2, 1])  # [B, C, H*W]

        correlation = tf.matmul(feature_A, feature_B)  # [B, H*W, H*W]
        correlation = tf.reshape(correlation, [b, h, w, h, w])  # [B, H, W, H, W]

        return correlation

# ... (rest of the code remains unchanged)

# Define the FeatureRegression class
class FeatureRegression(tf.keras.layers.Layer):
    def __init__(self):
        super(FeatureRegression, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(128, kernel_size=3, strides=1, padding='same', activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(64, kernel_size=3, strides=1, padding='same', activation='relu')
        self.conv3 = tf.keras.layers.Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu')
        self.conv4 = tf.keras.layers.Conv2D(6, kernel_size=3, strides=1, padding='same')

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        return x

# Assuming features_img1 and features_img2 are numpy arrays of shape [128, 16, 16]
# Convert features to tensors
feature_A = tf.convert_to_tensor(features_img1, dtype=tf.float32)
feature_B = tf.convert_to_tensor(features_img2, dtype=tf.float32)

# Add batch dimension
feature_A = tf.expand_dims(feature_A, axis=0)  # [1, 16, 16, 128]
feature_B = tf.expand_dims(feature_B, axis=0)  # [1, 16, 16, 128]

# Compute correlation map
correlation_layer = FeatureCorrelation()
correlation_map = correlation_layer(feature_A, feature_B)

print(correlation_map.shape)

# Initialize regression layer
regression_layer = FeatureRegression()

# Flatten and reshape correlation map
correlation_map_flattened = tf.reshape(correlation_map, [1, 256, 16, 16])  # [1, 256, 16, 16]

# Compute theta from correlation map
theta = regression_layer(correlation_map_flattened)

print(theta.shape)  # Should print (1, 6, 16, 16)


(1, 16, 16, 16, 16)
(1, 256, 16, 6)


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# Assume `img1_tensor` (PyTorch tensor) and `theta` (transformation parameters) are already defined

# Convert img1_tensor (PyTorch tensor) to NumPy array
img1_tensor_np = img1_tensor.numpy()  # Convert PyTorch tensor to NumPy array

# Convert NumPy array to TensorFlow tensor
img1_tensor_tf = tf.convert_to_tensor(img1_tensor_np, dtype=tf.float32)

# Add batch and channel dimensions to the TensorFlow tensor if not present
img1_tensor_tf = tf.expand_dims(img1_tensor_tf, axis=0)  # Add batch dimension
img1_tensor_tf = tf.expand_dims(img1_tensor_tf, axis=-1)  # Add channel dimension

# Define the transformation function
def apply_transform(image, theta):
    # Ensure theta is a 1D tensor with 6 elements
    if theta.shape != (6,):
        raise ValueError(f'Expected theta to be of shape (6,), but got {theta.shape}')

    # Define the affine transformation matrix for TensorFlow
    theta_tf = tf.reshape(theta, [6])  # Flatten theta to [6]

    # Apply the affine transformation
    transformed_image = tf.image.transform(
        image,
        theta_tf,
        interpolation='BILINEAR',
        fill_mode='CONSTANT',
        fill_value=0.0,
        output_shape=[256, 256]  # Change this as needed for your use case
    )

    return transformed_image

# Sample theta for an identity transformation (modify as needed)
theta = tf.constant([1, 0, 0, 0, 1, 0], dtype=tf.float32)  # Identity transformation

# Apply the transformation
transformed_img1 = apply_transform(img1_tensor_tf, theta)

# Convert tensor to NumPy array for visualization
transformed_img1_np = tf.squeeze(transformed_img1).numpy()

# Print tensor shapes for debugging
print(f'img1_tensor_tf shape: {img1_tensor_tf.shape}')
print(f'theta shape: {theta.shape}')
print(f'transformed_img1 shape: {transformed_img1.shape}')

# Visualize the transformed image
plt.imshow(transformed_img1_np, cmap='gray')
plt.title('Transformed Image')
plt.axis('off')  # Optional: Hide axis for better visualization
plt.show()


AttributeError: module 'tensorflow._api.v2.image' has no attribute 'transform'

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# Assume `img1_tensor` (PyTorch tensor) and `theta` (transformation parameters) are already defined

# Convert img1_tensor (PyTorch tensor) to NumPy array
img1_tensor_np = tf.squeeze(img1_tensor).numpy()
# Convert NumPy array to TensorFlow tensor
img1_tensor_tf = tf.convert_to_tensor(img1_tensor_np, dtype=tf.float32)

# Add batch and channel dimensions to the TensorFlow tensor if not present
# Add batch and channel dimensions to the TensorFlow tensor if not present
img1_tensor_tf = tf.expand_dims(img1_tensor_tf, axis=0)  # Add batch dimension
img1_tensor_tf = tf.expand_dims(img1_tensor_tf, axis=-1)  # Add channel dimension


# Define the transformation function
def apply_transform(image, theta):
    # Convert theta to 2D affine matrices for transformations
    theta = tf.reshape(theta, [-1, 2, 3])  # [1, 2, 3]

    # Create a grid for affine transformation
    grid = tf.image.affine_transform(
        image,
        theta[0, :, :],  # Use the first (and only) batch element's theta
        interpolation='BILINEAR',
        fill_mode='CONSTANT',
        fill_value=0.0,
        output_shape=[256, 256]
    )

    return grid

# Apply the transformation
transformed_img1 = apply_transform(img1_tensor_tf, theta)

# Convert tensor to NumPy array for visualization
transformed_img1_np = tf.squeeze(transformed_img1).numpy()

# Print tensor shapes for debugging
print(f'img1_tensor_tf shape: {img1_tensor_tf.shape}')
print(f'theta shape: {theta.shape}')
print(f'transformed_img1 shape: {transformed_img1.shape}')

# Visualize the transformed image
plt.imshow(transformed_img1_np, cmap='gray')
plt.title('Transformed Image')
plt.axis('off')  # Optional: Hide axis for better visualization
plt.show()


AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'squeeze'